# Informe Hito 2 Mineria de Datos - Grupo 16
## Integrantes:
- Constanza Dominguez ()
- Felipe Caballero ()
- Jose Pablo Canales ()
- Sebastian Salas ()
- Vicente Arce ()

## 1. Introduccion

En la actualidad, el universo de los videojuegos ha experimentado una expansión sin precedentes, con miles de títulos disponibles en múltiples plataformas y muchos más en desarrollo. Esta industria no solo ha crecido en termidos de producción y consumo, sino que tambien en la forma en que las personas interactúan con estos productos. El público que consume videojuegos abarca un amplio espectro demográfico, desde niños hasta adultos mayores, cada uno con percepciones, motivaciones y formas de relacionarse con los juegos significativamente distintas entre sí.

Dentro de las muchas prácticas asociadas en este entorno, surge una que ha ganado popularidad a través de los años, los speedruns. Esta actividad consiste, basicamente, en completar un videojuego en el menor tiempo posible siguiendo reglas de la misma comunidad. Lo que comenzó como un nicho se ha convertido en una forma competitiva de juego y un fenómeno cultural que con sus competiciones organizadas y transmisiones en directo logran captar la atención de miles de personas por todo el mundo.

Nuestro interés surgío, en un principio, por analizar un dataset centrado en los speedruns ya que ofrecía una buena calidad de datos y perspectiva social. Lo primero que se logró notar es que estos datos ofrecen una buena oportunidad para observar patrones de juego y rendimiento a través de distintos títulos, géneros y plataformas. Esto nos permitiría comprender mejor las dinamicas competitivas dentro de estas comunidades y extraer conocimiento valioso.

Sin embargo, al explorar más a fondo, logramos apreciar que el enfoque podía ser mucho más amplio y enriquecedor que solo limitarse a speedruns. Por ello decidimos hacer un estudio más amplio que abarcara el espectro de los videojuegos de manera general, y con esta información lograremos revelar aspectos interesantes de las relaciones sociales en la era digital. No obstante, esa práctica dejó en claro que los videojuegos no son solo una forma de entretenimiento para niños, sino que es un espacio mucho más complejo donde es posible desarrollar habilidades cognitivas y motoras significativas. Además, estudios recientes han demostrado el potencial de los videojuegos en areas de educación, creatividad, e incluso las mejoras en salud mental como el combate del deterioro cognitivo en adultos mayores.

En este informe exploraremos este dataset de manera sistemática, formulando hipótesis y extrayendo conocimiento significativo a partir de los registros de rendimiento en videojuegos, comprendiendo asi un fenomeno cultural contemporaneo.

## 2. Exploración de datos

Se presenta un análisis exploratorio del dataset de juegos de *HowLongToBeat*, centrado en entender las relaciones entre reseñas, tiempo de juego y variables complementarias. El dataset contiene 108 columnas con métricas de participación de usuarios, tiempos de completado, reseñas, plataformas y metadatos de juegos.

### Principales características del dataset

Se analizaron las principales características del dataset original, con los siguientes resultados:

| Métrica                                               | Valor     |
|-------------------------------------------------------|-----------|
| Total de juegos                                       | 147.474   |
| Juegos sin reviews                                    | 89.939    |
| Promedio de `game_review_score` (con reviews)         | 61.74     |
| Promedio de duración principal (`game_comp_main_avg`) | 2.45 horas |
| Promedio de duración entre modos de juego (`game_comp_all_avg`)          |5.23 horas|

### Estadísticas descriptivas relevantes

Se seleccionaron siete atributos relevantes basados en su potencial explicativo y cobertura de diferentes aspectos del dataset: calidad percibida, interacción de los usuarios, y duración del juego en horas.

| Variable               | Media  | Mediana | Desvío estándar | Mínimo | Máximo     |
| ---------------------- | ------ | ------- | --------------- | ------ | ---------- |
| `game_review_score`    | 24.26  | 0.0     | 32.38           | 0.0    | 100.00     |
| `review_review_count`  | 15.81  | 0.0     | 138.90          | 0.0    | 7439.00    |
| `game_count_replay`    | 1.86   | 0.0     | 22.34           | 0.0    | 2165.00    |
| `game_count_retired`   | 10.49  | 0.0     | 76.40           | 0.0    | 5227.00    |
| `game_count_comp`      | 49.52  | 1.0     | 476.26          | 0.0    | 36371.00   |
| `game_comp_all_avg`    | 5.23 h | 0.0 h   | 58.81 h         | 0.0 h  | 8105.08 h  |
| `game_invested_mp_avg` | 1.51 h | 0.0 h   | 51.77 h         | 0.0 h  | 10000.00 h |

De la tabla se puede ver la siguiente información:

- La mayoría de los juegos tienen un puntaje de reseña bajo o nulo (game_review_score y review_review_count con mediana = 0), lo que sugiere una alta cantidad de juegos poco evaluados por la comunidad.

- El promedio de game_count_comp es relativamente alto comparado con su mediana, lo que indica una distribución fuertemente sesgada: pocos juegos muy populares concentran la mayoría de las finalizaciones.

- Las métricas de tiempo (game_comp_all_avg y game_invested_mp_avg) presentan desviaciones estándar elevadas y valores máximos extremos, lo que indica gran variabilidad en duración de juegos, desde experiencias muy breves hasta compromisos prolongados.

- La baja mediana en casi todas las variables sugiere que la mayoría de los juegos tienen poca interacción o visibilidad, y solo un subconjunto reducido concentra el interés de los usuarios.

Este análisis justifica la necesidad de filtrar y escalar las métricas en etapas posteriores del procesamiento, especialmente para tareas de modelado o visualización.

### Análisis de atributos relacionados con "review"

Los atributos asociados a las reseñas de usuarios constituyen variables clave en este conjunto de datos. Estos campos, identificables por la raíz léxica "review", cuantifican tanto el volumen como la valoración de las evaluaciones recibidas por cada juego. El campo game_review_score representa el puntaje promedio de reseñas que tiene un juego, con valores que oscilan entre 0 y 100.

Adicionalmente, existen atributos con el formato review_X, donde X es un múltiplo de 5 entre 5 y 100 (por ejemplo, review_5, review_10, ..., review_100). Cada uno de estos atributos indica la cantidad de reseñas que otorgaron exactamente ese puntaje al juego. El campo review_review_count representa el número total de reseñas recibidas por el juego.

A continuación, se presenta una tabla que agrupa los juegos según intervalos de puntaje promedio (game_review_score) y muestra la cantidad de juegos que caen en cada intervalo. Además, se incluye la cantidad total de juegos que no han recibido ninguna reseña.

| #  | Intervalo de Puntaje | Cantidad de Juegos |
|----|----------------------|---------------------|
| 0  | 0-10                 | 90327               |
| 1  | 10-20                | 1117                |
| 2  | 20-30                | 1725                |
| 3  | 30-40                | 2385                |
| 4  | 40-50                | 4470                |
| 5  | 50-60                | 8526                |
| 6  | 60-70                | 14750               |
| 7  | 70-80                | 15085               |
| 8  | 80-90                | 6709                |
| 9  | 90-100               | 1547                |

Tambien se puede apreciar la siguiente distribución de los puntajes.

| Intervalo       | % Juegos | Interpretación               |
|-----------------|----------|------------------------------|
| 0 (Sin reseñas) | 61.17%    | Posible ruido para análisis  |
| 0-50            | 6.66%    | Recepción negativa           |
| 50-80           | 26.01%    | Recepción moderada           |
| 80-100          | 6.16%    | Excelente recepción          |

Se observa que existe una gran cantidad de juegos no han recibido ninguna review. Incluir estos juegos en el dataset puede resultar problemático si se desea realizar análisis que relacionen el puntaje promedio de reviews con otros atributos. Esto se debe a que, al no tener ninguna reseña, su puntaje promedio se representa como 0, lo cual no refleja una calificación real y puede introducir ruido o sesgos en el análisis. Por lo tanto, puede ser recomendable excluir estos registros en estudios donde las reviews son una variable clave. A continuación se puede apreciar la diferencia entre tener o no los juegos sin reviews.

<img src="img/juegos_vs_juegos_con_review.png" alt="juegos_vs_juegos_con_review" style="height:350px;"/>

Otro aspecto relevante del análisis es la relación entre la duración promedio de los juegos y su puntaje de reseñas. Para estudiar esta relación, se agruparon los juegos según su duración promedio (`game_comp_all_avg`), excluyendo aquellos que no cuentan con reseñas válidas. Esta decisión se justifica en que los juegos sin reseñas presentan un puntaje promedio de `0`, lo que no representa una calificación real y puede distorsionar los resultados.

La siguiente tabla muestra la distribución de juegos por intervalo de duración:


#### Cantidad de juegos por intervalo de duración (en horas)

| Intervalo de Duración | Cantidad de Juegos |
|------------------------|--------------------|
| 0-50 hrs               | 55,002             |
| 50-100 hrs             | 1,488              |
| 100-200 hrs            | 462                |
| 200-500 hrs            | 192                |
| 500-1000 hrs           | 67                 |
| >1000 hrs              | 54                 |


Como se observa, la gran mayoría de los juegos se concentran en el primer intervalo (menos de 50 horas), lo cual sugiere que el análisis visual con todos los datos puede estar fuertemente sesgado por outliers con duraciones extremadamente altas.

La siguiente visualización muestra dos gráficos de dispersión que relacionan la duración promedio de juego con su puntuación promedio de reseñas:

- El primero incluye todos los juegos válidos (sin reviews nulas),
- El segundo muestra el 95% inferior de los datos, eliminando outliers extremos.

<img src="img/puntuacion_vs_duracion_outlier.png" alt="puntuacion_vs_duracion_outlier" style="height:350px;"/>


Como puede apreciarse, al eliminar los valores atípicos (gráfico derecho), los puntos se agrupan de forma más compacta y se vuelve más evidente la estructura de los datos. Esta limpieza permite observar mejor la densidad y distribución de los juegos según su duración, revelando posibles patrones en su relación con la puntuación.

En particular, se nota que los juegos más extensos dentro del rango no atípico (hasta aproximadamente 100 horas) tienden a concentrarse en un rango de puntuaciones relativamente alto, aunque no necesariamente máximo. Esta observación sugiere una posible correlación entre mayor tiempo de juego y recepción más favorable por parte de los usuarios, aunque dicha correlación no es estrictamente lineal ni concluyente a simple vista.

Cabe destacar que en el gráfico sin outliers también se percibe una mayor variedad de puntuaciones en juegos de duración media (20–60 horas), lo que podría estar asociado a una mayor diversidad de géneros o experiencias ofrecidas en ese rango.

### Reducción de dimensiones

El conjunto de datos original contenía una gran cantidad de columnas, muchas de las cuales resultaban redundantes, irrelevantes o difíciles de interpretar. Con el objetivo de simplificar el análisis y mejorar la eficiencia de los modelos posteriores, se aplicó un proceso de reducción de dimensionalidad. Este proceso consistió en descartar o agrupar columnas según su utilidad y significado. Las decisiones principales fueron las siguientes:

- **Eliminación de columnas con significado desconocido:**
  Al haber sido recolectado mediante scraping, el dataset incluía 15 columnas cuyo propósito no pudo ser identificado. Estas columnas carecían de nombre descriptivo, contexto o documentación, lo que impedía su correcta interpretación y análisis. Por esta razón, se descartaron completamente.

- **Eliminación de columnas con información no procesable:**
  Se identificaron 6 columnas que contenían enlaces a imágenes, perfiles de usuario, páginas externas o texto libre sin una estructura estandarizada (como descripciones o títulos alternativos). Debido a que esta información no era útil en el contexto del análisis cuantitativo, también se eliminaron.

- **Agrupación de columnas de distribución de puntuaciones:**
  El dataset contenía 20 columnas del tipo `review_X`, donde `X` representa distintos umbrales de puntuación (5, 10, 15, ..., 100). Estas columnas representaban la cantidad de reseñas que asignaron una puntuación dentro de un cierto rango. Para reducir la dimensionalidad sin perder información significativa, se agruparon en cinco intervalos más amplios:

  - `review_20`: puntuaciones entre 1 y 20
  - `review_40`: puntuaciones entre 21 y 40
  - `review_60`: puntuaciones entre 41 y 60
  - `review_80`: puntuaciones entre 61 y 80
  - `review_100`: puntuaciones entre 81 y 100

  Esta transformación redujo las 20 columnas originales a solo 5, manteniendo la esencia de la distribución de puntuaciones y facilitando tanto la visualización como el procesamiento estadístico posterior.

En conjunto, estas acciones permitieron depurar significativamente el conjunto de datos, enfocando el análisis en variables relevantes y mejorando la interpretabilidad sin comprometer la calidad de la información.


## 3. Preguntas y problemas

A partir del set de datos y la exploración de este, se ha llegado a plantear las siguiente preguntas:

#### 1. ¿Qué características (género, tiempo de juego, rating) predicen mejor si un juego tendrá clasificación ESRB "M" (Maduro) frente a otras (ej: "T" para Teen)? ¿Puedo predecir cualquier clasificación de un juego basado en estos atributos? 

La clasificación de un juego determina el público objetivo de este. Entender qué características de un juego correlacionan con esto ayudará a la hora del desarrollo de este. Hipótesis: Juegos con etiquetas como Horror o Acción serán etiquetados como 'M'.  Así mismo, juegos con mayores tiempo de completitud serán de rating al menos M. Entonces, debería ser posible determinar el rating a partir de estas variables.

#### 2. ¿Podemos predecir el promedio de reviews de un juego en función de su tasa de abandono?

Los reviews de un juegos son un indicador de la recepción de este por parte de los jugadores, si un juego tiene altas tasas de abadono ¿repercute directamente en la review que reciba? Hipótesis: Los juegos con mayor tasa de abandono tendrán menores promedios de reviews que los con menos abandono.

#### 3. ¿Existen grupos de juegos con patrones similares que permitan agruparlos para recomendaciones?

Idintificar grupos entre los juegos para catalogarlos como 'similares' entre sí y poder ser recomendados juntos. Hipótesis: que se formen grupos entre juegos más rápidos de pasar y con menor rating, o juegos que tienen alto abandono y pertenecen a ciertos géneros.

#### 4. ¿Existen combinaciones de géneros o características (ej: "RPG + Open World") que correlacionen con tasas de abandono excepcionalmente altas/bajas?

Se busca encontrar características que influyan en el abandono de un juego. Hipótesis: Combinaciones como 'RPG + Open World' tendrán tasa de abandono significativamente mayor que 'RPG lineal', debido a la fatiga por contenido repetitivo. Otro como 'puzzle + indie' deben tener menos abandono quizás relacionado con su tiempo de completación.

## 4. Propuesta Metodológica Experimental

#### Pregunta - promedio de reviews a partir de tasa de abandono y rejugabilidad

Para responder la pregunta *¿Es posible predecir el promedio de reviews a partir de la tasa de abandono y rejugabilidad?*, planteamos un enfoque supervisado
Preprocesamiento
- Se mantiene el preprocesamiento mencionado anteriormente que ayuda a la limpieza de datos y reducción de dimensionalidad.
- Dejaremos solo las columnas que nos sirvan, en este caso, las de cantidad de abandonos, las de rejugabilidad, las de completitud, las totales, las cantidades de reviews y el promedio de reviews.
- Buscaremos un threshold para eliminar las filas que no contengan reviews y las que no tengan muy poca informacion sobre sus datos de abandonos y rejugabilidad.

Modelo y técnica de minería de datos a utilizar

Se utilizará un enfoque supervisado, ya que tenemos una variable objetivo conocida y queremos predecirla utilizando la tasa de abandono/rejugabilidad para ello se investigará la efectividad de
- Regresión lineal como modelo base para tener una primera aproximación a la relación entre las variables.
- Random Forest para identificar el caso en que se tenga una relación no lineal o interacciones entre las variables predictoras.

Criterio de evaluación de los resultados
- R² como medida de calidad global
- MSE como medida de error cuadrático medio para evaluar la precisión del modelo.


 #### Pregunta - Características para predecir clasificación


Para responder la pregunta *¿Qué características (género, tiempo de juego, rating) predicen mejor si un juego tendrá clasificación ESRB "M" (Maduro) frente a otras (ej: "T" para Teen)? ¿Puedo predecir cualquier clasificación de un juego basado en estos atributos?* utilizaremos un enfoque supervisado basado en clasificación.

Preprocesamiento adicional necesario
- Se eliminan del análisis aquellos valores nulos en game_rating_esrb y aquellos con "Unrated". Esto se debe realizar para tener datos útiles, no nulos para el análisis.
- Se estandariza la nomenclatura basada en intervalos, E, E+10, T, M, A fijos. Esto se realiza debido a que existen subcategorías como +3, +12, entre otros que se corresponden con E o con E+10 pero no están allí, similar para otros datos.
- Se define una columna binaria extra llamada "target_esrb_mature" donde se agruparán los juegos M con un 1 y los demás con 0. Esto ya que queremos que el modelo responda sí o no con respecto a una sola de las categorías.

Modelo y Técnica de minería de datos a utilizar
Dado que tenemos una pregunta de clasificación binaria, utilizaremos un enfoque supervisado utilizando, por un lado, árboles de decisión y random forest. Esto debido a que son algoritmos interpretables, permiten identificar la importancia de las variables predictoras y manejan bien tanto variables categóricas como continuas sin requerir mucho preprocesamiento. Además, la random forest ayuda a reducir el sobreajuste al promediar múltiples árboles.


Criterio de evaluación de los resultados
Para evaluar el desempeño de los modelos se utilizarán las siguientes métricas:
- Accuracy: para observar qué tan bien clasifica en general.

- Precisión y recall para la clase 'M': dado que puede haber desbalance en las clases, estas métricas nos permiten evaluar específicamente qué tan bien se identifican los juegos maduros.

- Matriz de confusión: para visualizar los verdaderos positivos, falsos positivos, verdaderos negativos y falsos negativos.

- Curva ROC y AUC: para medir la capacidad discriminativa del modelo entre las clases.

Con base en estas métricas, podremos identificar qué características (por ejemplo, ciertos géneros, tiempos de juego largos o ratings de usuarios elevados) son más comunes entre los juegos clasificados como "M", y si es viable predecir la clasificación ESRB únicamente con atributos como género, tiempo de juego y rating general del juego.


#### Pregunta - Combinaciones de géneros o características que correlacionen con tasas de abandono expecionalmente bajas/altas?

Para responder la pregunta *¿Existen combinaciones de géneros o características (ej: "RPG + Open World") que correlacionen con tasas de abandono excepcionalmente altas/bajas?*, aplicaremos un enfoque no supervisado basado en clustering

Preprocesamiento necesario
- Convertir game_profile_genre en variables binarias, permitiendo detectar combinaciones como "RPG + Open World"
- Definición de tasa de abandono: $\text{Tasa de abandono} = \frac{\text{game\_count\_retired}}{\text{game\_count\_total}}$
- Filtrado de juegos con baja cantidad de usuarios: Para evitar sesgos por juegos con pocos registros, se puede eligir un umbral mínimo de 10 usuarios.
- Agrupación de características adicionales:
Incluir variables binarias para: plataforma principal (game_profile_platform), modos de juego (game_comp_lvl_sp, co, mp), tipo de juego (game_game_type). Esto permitirá generar un análisis más granular y con mayor control en los datos.

Modelo y Técnica de minería de datos a utilizar
Se aplicará un enfoque no supervisado sobre las combinaciones de géneros y características jugables para encontrar patrones asociados a tasas de abandono. Es no supervisado dado que no conocemos combinaciones de antemano que aseguren una correlacion con tasas de abandono.
- Se utilizará K-Means para evaluar agrupaciones homogéneas en torno a tasas de abandono similares.
- Se probará también con DBSCAN como técnica alternativa, especialmente útil si los grupos son irregulares o no esféricos.

Criterio de evaluación de los resultados
- Se utilizará el Silhouette Score para evaluar la coherencia interna de los clusters formados.

#### Pregunta - Grupos de juegos con patrones similares para recomendaciones.

Para responder la pregunta *¿Existen grupos de juegos con patrones similares que permitan agruparlos para recomendaciones?*, aplicaremos un enfoque no supervisado basado en clustering

Preprocesamiento necesario adicional
- Se mantiene el preprocesamiento mencionado anteriormente que ayuda a la limpieza de datos y reducción de dimensionalidad.

Modelo y Técnica de minería de datos a utilizar
- Se utilizará un enfoque no supervisado basado en clustering, esto porque no contamos con etiquetas predefinidas de "juegos recomendables juntos". Se evaluará con el algoritmo de k-means esperando una respuesta de grupos homogéneos, comparar con varios valores de k. Además se comparará con DBSCAN, que encontrará un número de clusters y permitirá contrastar en caso de que los grupos no sean homogéneos ni con una forma esférica.

Criterio de evaluación de los resultados
- Se utilizará Silhouette Score para ver que tan separados y cohesionado está un punto en su cluster y así se comparará para los distintos valores de k en k-means. Esto permitirá escoger un valor de k apropiado y que los clusters generados correspondan efectivamente a grupos cohesionados y separados.
- Se utilizará una evaluación semántica basada en el contexto, sobre todo se pretende evaluar aquí que tenga sentido que ciertos juegos estén agrupados y que por ende compartan algunos atributos como género, promedio de reviews, promedio en tiempo de completitud, entre otros.


### 5. Resultados Preliminares

#### Análisis Preliminar de los Datos

Durante la etapa de exploración inicial se detectó que un gran número de juegos no cuenta con reviews registradas. Esto representa un problema, ya que la variable que buscamos predecir es precisamente el promedio de puntuaciones obtenidas a través de estas reviews.

Además, no todos los jugadores registran el abandono o cuando rejuegan algun juego. Esto implica que trabajar con juegos que tienen muy pocos registros totales pueden introducir ruido en el análisis. Por ejemplo, un juego podría parecer que tiene baja tasa de abandono simplemente porque nadie registró haberlo abandonado, o parecer rejugable por un único registro. En juegos con una base de jugadores muy reducida, estas interpretaciones pueden ser erróneas.

Debido a lo anterior, fue necesario establecer **umbrales mínimos (thresholds)** para considerar un juego en el análisis. Estos thresholds fueron definidos inicialmente desde un punto de vista exploratorio, basándose en la distribución de reseñas. Se probaron distintos valores para evaluar su impacto en la calidad de los datos.

<img src="img/review-distribution1.png" alt="review-dist1" style="height:350px;"/>

#### Construcción del Conjunto de Datos

Una vez definidos los thresholds, se creó un nuevo dataframe filtrado que solo contiene los juegos que superan dichos umbrales. Este nuevo conjunto de datos conserva únicamente las columnas relevantes: número de usuarios que completaron, abandonaron, rejugaron, reseñaron, el total de usuarios por juego, y el promedio de reviews.

A partir de estas columnas, se construyeron nuevas métricas:

- **Tasa de abandono**: usuarios que abandonaron dividido por el total de usuarios.
- **Tasa de rejugabilidad**: usuarios que rejugaron dividido por el total de usuarios.
- **Engagement**: una nueva métrica definida como la proporción entre los usuarios que completaron o rejugaron un juego respecto a la suma de estos y los que abandonaron.

Se observó que estas tasas entregaban mejores metricas en los modelos al calcularlas respecto del total de usuarios, en lugar de tomar como base únicamente los completados. Ademas, agregar la métrica de engagement mostró una mejora significativa en el valor de R², lo que sugiere que aporta valor predictivo.

#### Análisis de Correlaciones

Con el dataframe ya procesado, se definieron las variables:
- `X`: compuesto por las tres tasas mencionadas.
- `y`: correspondiente al promedio de reviews por juego.

Se realizaron visualizaciones para investigar patrones entre las tasas y la variable objetivo, así como una matriz de correlación. Estas herramientas confirmaron la existencia de correlaciones entre las tasas y el promedio de reviews, siendo **engagement** la más clara.
<p float="left">
    <img src="img/scatter-rates1.png" alt="scatter-rates" style="height:350px;"/>
    <img src="img/correlation-matrix.png" alt="juegos_vs_juegos_con_review" style="height:350px;"/>
</p>

#### Entrenamiento y Evaluación de Modelos

El conjunto de datos fue dividido en entrenamiento y prueba, utilizando una proporción 80/20. Se entrenaron distintos modelos para predecir el promedio de reviews:

- **Regresión lineal**
- **Random Forest**
- Ambos modelos también fueron tambien evaluados utilizando **Polynomial Features**

| Modelo                     |     R²     |    MSE     |
|---------------------------|------------|------------|
| Linear Regression         |  0.426646  | 53.414461  |
| Polynomial Regression     |  0.483635  | 48.105336  |
| Random Forest             |  0.439527  | 52.214458  |
| Polynomial + Random Forest|  0.443171  | 51.874948  |

#### Conclusiones Preliminares

Los resultados indican que, si bien las tasas de abandono, rejugabilidad y engagement muestran cierta correlación con el promedio de reviews, **no es posible predecirlo con gran precisión utilizando estos modelos y características**. El mejor desempeño obtenido fue un R² de **0.45** y un error estándar mínimo de **48**, lo cual es considerable si se toma en cuenta que las puntuaciones están en una escala de 0 a 100.

#### Posibles Mejoras

Para futuros experimentos se proponen las siguientes mejoras:

- Ajustar de forma más precisa los thresholds, filtrando con base en la representatividad real de los registros.
- Explorar la incorporación de nuevas variables relacionadas al engagement u otras métricas del juego.
- Evaluar otros modelos de entrenamiento que puedan capturar relaciones no lineales o más complejas en los datos.

Estos ajustes permitirán refinar la capacidad predictiva del modelo y acercarse de mejor forma a la pregunta de investigación planteada.

## Anexo
- Link a codigo para la exploracion de datos: https://github.com/SebaSalasLvd/proyecto-mineria/blob/main/hito-1/Exploracion_Datos_Hito_1.ipynb
- Link a codigo para los resultados preliminares: https://github.com/SebaSalasLvd/proyecto-mineria/blob/main/hito-1/resultados_prelim.ipynb